In [1]:
%%capture
# Uncomment if you haven't these packages
%pip install --upgrade accelerate peft trl huggingface_hub
%pip install "transformers==4.38.2" # Bug occured in v4.39.1 - AttributeError: 'torch.dtype' object has no attribute 'itemsize'

In [2]:
from os import path,chdir
import sys
chdir(path.dirname(path.realpath(sys.argv[0]))) # change working directory to script location

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from scripts.improve_result import improve_result
from huggingface_hub import login

In [4]:
%env CUDA_VISIBLE_DEVICES=""

env: CUDA_VISIBLE_DEVICES=""


In [5]:
login()

In [6]:
model_name = "Tony177/codellama-13b-dockerfile-generation"

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cpu",
)
model.config.use_cache = False
model.config.pretraining_tp = 1 # Setting config.pretraining_tp to a value different than 1 will activate the more accurate but slower computation of the linear layers, which should better match the original logits.
model.enable_input_require_grads() # Warning about gradients during generation

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [8]:
# Load the tokenizer from Hugginface and set padding_side to “right” to fix the issue with fp16
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="cpu")
tokenizer.add_special_tokens({'pad_token': '[PAD]'}) # Add padding token to tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [9]:
def generate_text(tokenizer, model, prompt: str) -> str:
    prompt += ". Output must be in valid Dockerfile format. Output must include only dockerfile commands."
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cpu") # Added last part to avoid crash to KeyError: 'shape'
    # beam-search multinomial sampling if num_beams>1 and do_sample=True
    gen_tokens = model.generate(input_ids, max_new_tokens=512, num_beams=5, no_repeat_ngram_size=2, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    result = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0] # One element list, just the response
    return improve_result(prompt, result)

In [10]:
def generate_text_INST(tokenizer, model, prompt: str) -> str:
    prompt = "<s>[INST] " + prompt + ". Output must be in valid Dockerfile format. Output must include only dockerfile commands. [/INST]"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cpu") # Added last part to avoid crash to KeyError: 'shape'
    # beam-search multinomial sampling if num_beams>1 and do_sample=True
    gen_tokens = model.generate(input_ids, max_new_tokens=512, num_beams=5, no_repeat_ngram_size=2, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    result = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0] # One element list, just the response
    return improve_result(prompt, result)

In [ ]:
print(generate_text(tokenizer, model, "Generate a dockerfile of Wordpress 5.7"))

In [ ]:
print(generate_text_INST(tokenizer, model, "Generate a dockerfile of Wordpress 5.7"))

In [ ]:
print(generate_text(tokenizer, model, "Generate a dockerfile of Python 3.7"))

In [ ]:
print(generate_text_INST(tokenizer, model, "Generate a dockerfile of Python 3.7"))

In [ ]:
print(generate_text(tokenizer, model, "Generate a Dockerfile of Ruby 3.2.1"))

In [ ]:
print(generate_text_INST(tokenizer, model, "Generate a Dockerfile of Ruby 3.2.1"))